# TODO

https://towardsdatascience.com/winning-blackjack-using-machine-learning-681d924f197c

1. Andmestiku puhastus, ainult need asjad mida vaja on.
2. Sorteerida ära terve andmestiku peale pärast esimese kahe kaardi andmist kas käsi on soft-hand, hard-hand või paar (nagu need tabelid üleval antud lehel on). Hard-handid saame nii kätte, et kus kaks kaarti pole kas paar või üks kahest kaardist ei ole 
3. Nüüd kui meil on ära sorteeritud käed, siis iga käega peaksime välja arvutama (I assume) võiduprotsendi järgi kas me peame hittima, stay või panust duubeldama. 
4. Ja nii arvutame välja kõikide juhtudega ehk kui sul on kolm kaarti käes, neli kaarti jne.

In [15]:
import pandas as pd
import warnings

data = pd.read_csv("blkjckhands.csv", index_col=False)
#display(data)
data = data.drop(columns=['Unnamed: 0', 'PlayerNo', 'dealcard2', 'dealcard3', 'dealcard4', 'dealcard5', 'plybustbeat', 'dlbustbeat','plwinamt', 'dlwinamt'])
data

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum
0,7,10,0,0,0,17,10,18,nowin,Loss,17
1,10,9,0,0,0,19,10,18,nowin,Win,19
2,9,8,0,0,0,17,10,18,nowin,Loss,17
3,2,10,0,5,0,17,10,18,nowin,Loss,12
4,10,2,0,5,0,17,10,18,nowin,Loss,12
...,...,...,...,...,...,...,...,...,...,...,...
899995,10,7,0,0,0,17,3,18,nowin,Loss,17
899996,6,1,10,0,0,17,3,18,nowin,Loss,7
899997,4,2,9,0,0,15,3,18,nowin,Loss,6
899998,9,10,0,0,0,19,3,18,nowin,Win,19


In [16]:
cond = (data.card3 == 0) & (data.card4 != 0)
data.loc[cond, ['card3', 'card4']] = data.loc[cond,['card4', 'card3']].values



#154980 - kus on kolmas kaart on õiges kohas
#190401 - kus on kolmas kaart vales kohas
#35603 - kus on kolmas ja neljas kaart vales kohas

In [17]:
cond2 = (data.card3 != 0) & (data.card4 == 0) & (data.card5 != 0)
data.loc[cond2, ['card4', 'card5']] = data.loc[cond2, ['card5', 'card4']].values

data[(data["card3"] != 0) & (data["card4"] == 0) & (data["card5"] != 0)]

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum


In [18]:
#Sorteerisin käed vastavalt esimese kahe kaardi järgi, sealt vastavalt saame teha maatrixi siis otsustest based on win rate
#Lisaks peab jälgima seda, et meil on andmeid kus on kolmas kaart 0 aga neljas ja viies on hit. Need peame lugema pmst hittideks,
#lihtsalt 

#Ehk siis me arvutame iga käe summa järgi ning kas sellega võideti või mitte. Lisaks vaatame dealeri kaarti. Umbes kuidagi selline on
#arvutus:
# 1. Näiteks hard_hands puhul võtame summa nagu näiteks 10. 
# 2. Loeme kõik 10ned kokku. Eraldame kõik diileri kaardi juhud ning võtame näiteks kus on sinu kaartide summa on 10 ning diileri summa on 2
# 3. Loeme kokku win% kõikidel kätel kus hitidakse ning kus ei hitida. (not sure how to make this work yet).
# 4. Paneme sinu tehtud maatriksisse lõpuks 0 asemele kas me hitime selle summaga ning selle dealeri handiga või standime.

#Sarnast algoritmi saab kasutada ka paaride ning soft_handide korral ehk me võime kasutada paaride korral näiteks alguses nn paaride maatriksit,
#ning näiteks teeme statistika selle põhjal kas splittida või mitte. Sealt me edasi mõtlema ei pea, kuna edasi läheb ta nkni otsustab kas tema käsi
#on hard_hand või mis iganes. Sama ka soft_handide põhjal.

pairs = data[data["card1"] == data["card2"]].copy()
soft_hands = data[(data["card1"] == 1) | (data["card1"] == 11) | (data["card2"] == 1) | (data["card2"] == 11)].copy()
hard_hands = data[(data["card1"] != data["card2"]) & ((data["card1"] != 1) & (data["card1"] != 11) & (data["card2"] != 1) & (data["card2"] != 11))].copy()

pairs[(pairs["card1"] == 2)]
Acesubset = data[(data["card1"]==1)|(data["card1"]==11)|(data["card2"]==1)|(data["card2"]==11)]
Acesubset

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum
5,6,1,8,10,0,25,10,18,nowin,Loss,7
14,5,1,10,0,0,16,4,26,nowin,Win,6
34,2,11,4,0,0,17,3,15,nowin,Win,13
44,9,11,0,0,0,20,5,17,nowin,Win,20
46,2,11,7,0,0,20,5,17,nowin,Win,13
...,...,...,...,...,...,...,...,...,...,...,...
899974,11,1,9,0,0,21,6,19,nowin,Win,12
899982,10,11,0,0,0,21,6,26,Win,Win,21
899984,11,10,0,0,0,21,6,26,Win,Win,21
899985,10,11,0,0,0,21,6,26,Win,Win,21


Otsustuse dataframe on vaja panna paika siis nende kriteerimite alusel:
1. Row tähistab handi väärtust ja veerg tähistab dealeri esimest kaarti. Me peame row väärtuseks võtma ka need handid kus ntks alles viies kaart oli see mis summaks andis 20 ehk me itereerime iga rida kus ka kaartide vaheväärtus oli otsitav summa. Ühesõnaga ütleme, et me otsime hande kus kaartide summaks oli 18, st et me vaatame hande kus esimese kahe kaardi, esimese kolme kaari, esimese nelja kaardi... jne summa oli 18 ja kas siis oli HIT v oli STAND ja kas käsi lõpuks võideti või ei.

2. Me loeme HITID ja STANDID kokku ja vaatame kumma puhul suurem võidu % oli ja kanname vastavalt siis kummagi tabelisse.

In [19]:
veerud = {'2':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']}
hardHands = pd.DataFrame(veerud, index=['20','19','18','17','16','15','14','13','12','11','10','9','8','7','6','5'])

In [20]:
veerud = {'2':['0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0']}

softHands = pd.DataFrame(veerud, index=['A-9', 'A-8', 'A-7', 'A-6', 'A-5', 'A-4', 'A-3', 'A-2'])


veerud = {'2':['0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0']}
pairTable = pd.DataFrame(veerud, index=['A-A', 'T-T', '9-9', '8-8', '7-7', '6-6', '5-5', '4-4','3-3','2-2'])

In [21]:
#Teeme ühe korra arvutuse läbi
#Võtame kaartide summaks 5 ja dealeri uphandiks 2
#temp = hard_hands[(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] + hard_hands["card5"] == 5)].copy()
def hitvstand(summa,dcard, hand):
    if(hand == 'hard'):
        summa = int(summa)
        if(dcard == "A"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & ((hard_hands["dealcard1"] == 1) | (hard_hands["dealcard1"] == 11))
                ].copy()
        elif(dcard == "T"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == 10)
                ].copy()
        else:
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == int(dcard))
                ].copy()


        standwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]==0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]==0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

        hitwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]!=0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
        hitwinNr = len(hitwin.index)
        standwinNr = len(standwin.index)
        
        if(hitwinNr > standwinNr):
            return 'H'
        else:
            return 'S'
    
    if(hand == 'soft'):
        teinekaart = summa.split("-")
        summa1 = 1 + int(teinekaart[1])
        summa2 = 11 + int(teinekaart[1])
        
        if(dcard == "A"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & ((soft_hands["dealcard1"] == 1) | (soft_hands["dealcard1"] == 11))
            ].copy()
        elif(dcard == "T"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == 10)
            ].copy()
        else:
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == int(dcard))
            ].copy()
            
        hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwinNr = max(len(hitwin1.index), len(hitwin2.index))
        standwinNr = max(len(standwin1.index), len(standwin2.index))
        

        if(hitwinNr > standwinNr):
            return 'H'
        else:
            return 'S'
    
    if(hand=="pair"):
        card1 = summa[2]
        card2 = summa[0]
        if((card1=="A") & (card2=="A")):
            card11=1
            card12=11
            card21=1
            card22=11
            summa1=2
            summa2=12
            if(dcard == "A"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
            hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                          &
                          ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                         ].copy()
            standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                            &
                            (temp["card3"] == 0)
                           ].copy()
            hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                          &
                          ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                         ].copy()
            standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                            &
                            (temp["card3"] == 0)
                           ].copy()
            
            splitwin = Acesubset[((Acesubset["winloss"]=="Win") & (Acesubset["card1"]==card11))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card1"]==card12))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card2"]==card21))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card2"]==card22))].copy()

            hitwinprecent = '%.2f' % (max(len(hitwin1.index), len(hitwin2.index))/len(temp.index))
            standwinprecent = '%.2f' % (max(len(standwin1.index), len(standwin2.index))/len(temp.index))
            splitwinprecent = '%.2f' % ((len(splitwin.index)/len(Acesubset.index)))
            
            #return str("HIT: "+str(hitwinprecent)+ " STAND:" + str(standwinprecent) +"  SPLIT: "+str(splitwinprecent)) 
            
            if (splitwinprecent >= hitwinprecent) and (splitwinprecent >= standwinprecent):
                return "P"
            elif (hitwinprecent >= splitwinprecent) and (hitwinprecent >= standwinprecent):
                return "H"
            else:
                return "S"
            
        
        else:
            if((card1=="T") & (card2=="T")):
                summa=20
                card1=10
                card2=10
            else:
                card1=int(card1)
                card2=int(card2)
                summa=card1+card2
            if(dcard == "A"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa) 
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
            standwin = temp[(temp["winloss"]=="Win") & ((temp["card3"]==0)
                          |
                        (temp["card3"]!=0) & (temp["card4"]==0)
                         |
                         (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

            hitwin = temp[(temp["winloss"]=="Win")& ((temp["card3"]!=0)
                          |
                        (temp["card3"]!=0) & (temp["card4"]!=0)
                         |
                         (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
            
            smaller_data=data[(data["card1"]==card1)|(data["card2"]==card2)]
            splitwin = smaller_data[((smaller_data["winloss"]=="Win") & (smaller_data["card1"]==card1))|
                                    ((smaller_data["winloss"]=="Win") & (smaller_data["card2"]==card2))]
            splitwinprecent =  '%.2f' % ((len(splitwin.index)/len(smaller_data.index)))
            hitwinprecent =  '%.2f' % (len(hitwin.index)/len(temp.index))
            standwinprecent = '%.2f' % (len(standwin.index)/len(temp.index))

            
            
            #return str("HIT: "+str(hitwinprecent)+ "  STAND:" + str(standwinprecent) +"  SPLIT: "+str(splitwinprecent)) 
            
            if (splitwinprecent >= hitwinprecent) and (splitwinprecent >= standwinprecent):
                return "P"
            elif (hitwinprecent >= splitwinprecent) and (hitwinprecent >= standwinprecent):
                return "H"
            else:
                return "S"
              
    
        
        
    


In [22]:
Columns = list(hardHands.columns)
Rows = list(hardHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'hard')
        hardHands.iloc[i][j] = taht
        
hardHands

,2,3,4,5,6,7,8,9,T,A
20,S,S,S,S,S,S,S,S,S,S
19,S,S,S,S,S,S,S,S,S,S
18,S,S,S,S,S,S,S,S,S,S
17,S,S,S,S,S,S,S,S,S,S
16,S,S,S,S,S,H,H,H,H,H
15,S,S,S,S,S,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H
12,H,H,S,S,S,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H


In [23]:
Columns = list(softHands.columns)
Rows = list(softHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], "soft")
        softHands.iloc[i][j] = taht
softHands

,2,3,4,5,6,7,8,9,T,A
A-9,S,S,S,S,S,S,S,S,S,S
A-8,S,S,S,S,S,S,S,S,S,S
A-7,S,S,S,S,S,S,S,S,S,S
A-6,H,H,H,H,H,H,H,H,H,H
A-5,H,H,H,H,H,H,H,H,H,H
A-4,H,H,H,H,H,H,H,H,H,H
A-3,H,H,H,H,H,H,H,H,H,H
A-2,H,H,H,H,H,H,H,H,H,H


In [24]:
Columns = list(pairTable.columns)
Rows = list(pairTable.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'pair')
        pairTable.iloc[i][j] = taht
        
pairTable

,2,3,4,5,6,7,8,9,T,A
A-A,P,P,P,P,P,P,P,P,P,P
T-T,S,S,S,S,S,S,S,S,S,S
9-9,S,S,S,S,S,S,P,P,P,P
8-8,P,P,P,S,P,P,P,P,P,P
7-7,S,P,S,S,S,P,P,P,P,P
6-6,H,P,S,S,S,H,P,P,P,P
5-5,H,H,H,H,H,H,H,H,H,P
4-4,H,H,H,H,H,H,P,P,P,P
3-3,H,H,H,H,H,P,P,P,P,P
2-2,H,H,H,H,H,H,P,P,P,P


In [25]:
str(pairTable.loc['2-2']['2'])

'H'

In [33]:
from random import *

def Blackjackplayer(iteratsioonide_arv):
    df = pd.DataFrame(columns=['card1','card2','card3','card4','card5','dealcard1','dealcard2','dealcard3','dealcard4','dealcard5',"winloss"])
    
    for i in range(iteratsioonide_arv):
        
        playercards=[] #playeri hand list
        dealercards=[] # diileri handi list
        Endgame=False
        Stand=False
        handtype= ""
        plwinloss=""
        dwinloss=""
        stand=False
        plendgame=False
        for i in range(5): #genereerib randomiga diilerile ja playerile kaardid ette. prg 5 sest esialgses datas ka nii.
            playercards.append(randint(1,11))
            dealercards.append(randint(1,11))
        dealcard = dealercards[0] #diileri n2htav kaart
        if(dealcard == 11 | dealcard == 1):
            dealcard = 'A'
        elif(dealcard == 10):
            dealcard = 'T'

        if(playercards[0]==playercards[1]): # kui paar on 
            handtype="pair"
            if(playercards[0]==11 | playercards[0]==1):
                hand="A-A"
            if(playercards[0]==10):
                hand="T-T"
            else:
                hand=str(str(playercards[0])+"-"+str(playercards[1])) #Handi formatimine tabeli kasutamise jaoks

        elif((playercards[0]==1) | (playercards[1]==1)|(playercards[0]==11) | (playercards[1]==11)): #kui softhand on
            handtype="soft"
            if(playercards[0]==1 | playercards[0]==11):
                hand=str("A-"+str(playercards[0])) #Selle jaoks, et teada kumb pool esialgsest kahest kaardist Ace on.
            else:
                hand=str("A-"+str(playercards[1]))

        else: #kui hardhand on
            handtype="hard"
            hand=playercards[0]+playercards[1]


        ##VAJA JUURDE MÕELDA, ET KUIDAS LAHENDADA OLUKORDA KUI SAAME ÄSSA

        haveAce = False
        lowSoft = False

        while((plendgame==False) & (stand==False)):
            iteraator=2 #mille järgi järgmise kaardi võtab listist et summale liita
            if(handtype=="hard"):
                if(hand==21):
                    plwinloss="Win"
                    plendgame=True
                if(hardHands.loc[str(hand)][str(dealcard)] == "H"):
                    if(lowSoft): #Softhandist hardhandiks ning jätame väiksema käe meelde.
                        if(playercards[iteraator] == 1 | playercards[iteraator] == 11): 
                            if(hand1 > 10):
                                hand1 += 1
                            else:
                                hand1 += 11
                        else:
                            hand1 += playercards[iteraator]
                    if((playercards[iteraator] == 1 | playercards[iteraator] == 11)): #Kui saame ässa
                        if(hand > 10 and not haveAce):
                            hand += 1
                        else:
                            if(haveAce):
                                if(hand > 10):
                                    hand += 1
                                    if(lowerHand > 10):
                                        lowerHand += 1
                                    else:
                                        lowerHand += 11

                                else:
                                    lowerHand = hand + 1
                                    hand += 11
                                    haveAce = True
                            else:
                                hand+=playercards[iteraator]
                    iteraator+=1
                    if(hand>21):
                        if(lowAce and hand1 < 21):
                            hand = hand1
                        elif(haveAce and lowerHand < 21):
                            hand = lowerHand
                        else:
                            plwinloss="Loss"
                            plendgame=True
                elif(hardHands.loc[str(hand)][str(dealcard)] == "S"):
                    stand=True

            if(handtype=="soft"):
                if(hand[2:4]==str(10)):
                    plwinloss="Win"
                    plendgame=True
                elif(softHands.loc[str(hand)][str(dealcard)] == "H"):
                    if(playercards[iteraator] == 1 | playercards[iteraator] == 11):
                        hand1 = 1 + int(hand[2:4]) 
                        hand2 = 11 + int(hand[2:4]) + 1
                        if(hand1 + 11 > 21):
                            hand1 += 1
                        else:
                            hand1 += 11
                    else:
                        hand1 = 1 + int(hand[2:4]) + playercards[iteraator]
                        hand2 = 11 + int(hand[2:4]) + playercards[iteraator]
                    iteraator += 1
                    handtype = 'hard'
                    if(hand1 > 21 and hand2 > 21):
                        plwinloss="Loss"
                        plendgame=True
                    elif(hand2 > 21 and hand1 < 21):
                        hand = hand2
                    elif(hand1 > 21 and hand2 < 21):
                        hand = hand1
                    else:
                        hand = hand2
                        lowSoft = True
                elif(softHands.loc[str(hand)][str(dealcard)] == "S"):
                    stand = True
                    hand = 11 + int(hand[2:4])
            #if(handtype=="pair"):
                #if




        while(dendgame==False & dstand==False):
            iteraator=2
            dsumma=dealercards[0]+dealercards[1]
            if(dsumma==21):
                dwinloss="Win"
                dendgame=True
            if(dealercards[0]==11 | dealercards[1]==11):
                if(dsumma>21):
                    dsumma-=10
            if(dsumma<17):
                dsumma+=dealercardsl[iteraator]
                iteraator+=1
                if(dsumma>21):
                    dwinloss="Loss"
                    dendgame=True
            else:
                dstand=True

        if(plwinloss=="" & dwinloss==""):
            if(dsumma>summa):
                plwinloss="Loss"
                dwinloss="Win"
            if(summa==dsumma):
                plwinloss="Push"
                dwinloss="Push"
            else:
                plwinloss="Win"
                dwinloss="Loss"
        if(plwinloss=="Win" & dwinloss=="Loss"):
            
            df = df.append({'card1': playercards[0],
               "card2":playercards[1],
               "card3":playercards[2],
               "card4":playercards[3],
               "card5":playercards[4],
               "dealcard1":dealercards[0],
               "dealcard2":dealercards[1],
               "dealcard3":dealercards[2],
               "dealcard4":dealercards[3],
               "dealcard5":dealercards[4],
               "winloss":plwinloss}, ignore_index=True)
            
        if(plwinloss=="Win"& dwinloss =="Win"):
            plwinloss=="Push"
            dwinloss=="Push"
            df = df.append({'card1': playercards[0],
               "card2":playercards[1],
               "card3":playercards[2],
               "card4":playercards[3],
               "card5":playercards[4],
               "dealcard1":dealercards[0],
               "dealcard2":dealercards[1],
               "dealcard3":dealercards[2],
               "dealcard4":dealercards[3],
               "dealcard5":dealercards[4],
               "winloss":plwinloss}, ignore_index=True)
        else:
            plwinloss="Loss"
            dlwinloss="Win"
            df = df.append({'card1': playercards[0],
                           "card2":playercards[1],
                           "card3":playercards[2],
                           "card4":playercards[3],
                           "card5":playercards[4],
                           "dealcard1":dealercards[0],
                           "dealcard2":dealercards[1],
                           "dealcard3":dealercards[2],
                           "dealcard4":dealercards[3],
                           "dealcard5":dealercards[4],
                           "winloss":plwinloss}, ignore_index=True)
    
    #Loogika peaks nüüd vist olemas olema. Returnid on vaja ümber teha nii, et nad kuhugi dataframe-i kirjutaks.
    return df;
        
        
            
    #todo while loop diileri jaoks
    #l6pus võrdled hande
                    
                    
            
        
        
    
        
        
    

In [34]:
display(Blackjackplayer(100))

KeyError: '1'